In [ ]:
from google.colab import files
import pandas as pd

# Upload the CSV file
uploaded = files.upload()

Saving mtsamples.csv to mtsamples.csv


In [ ]:
clinical_data = pd.read_csv("mtsamples.csv")
clinical_data.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [ ]:
category_count = clinical_data['medical_specialty'].value_counts()
print(category_count)

 Surgery                          1103
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        372
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  230
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Obstetrics / Gynecology           160
 Urology                           158
 Discharge Summary                 108
 ENT - Otolaryngology               98
 Neurosurgery                       94
 Hematology - Oncology              90
 Ophthalmology                      83
 Nephrology                         81
 Emergency Room Reports             75
 Pediatrics - Neonatal              70
 Pain Management                    62
 Psychiatry / Psychology            53
 Office Notes                       51
 Podiatry                           47
 Dermatology                        29
 Dentistry                          27
 Cosmetic / Plastic Surge

In [ ]:
data_categories  = clinical_data.groupby(clinical_data['medical_specialty'])
data_categories.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."
...,...,...,...,...,...,...
4988,4988,Autopsy - Homicide - Blunt force cranial trauma,Autopsy,Autopsy - 1,"EXTERNAL EXAMINATION - SUMMARY,The body is pre...",NaN
4993,4993,"Chronic glossitis, xerostomia, probable envir...",Allergy / Immunology,Evaluation of Allergies,"HISTORY:, A 55-year-old female presents self-...","allergy / immunology, chronic glossitis, xeros..."
4994,4994,Patient having severe sinusitis about two to ...,Allergy / Immunology,Chronic Sinusitis,"HISTORY:, I had the pleasure of meeting and e...",NaN
4995,4995,This is a 14-month-old baby boy Caucasian who...,Allergy / Immunology,Kawasaki Disease - Discharge Summary,"ADMITTING DIAGNOSIS: , Kawasaki disease.,DISCH...","allergy / immunology, mucous membranes, conjun..."


In [ ]:
filtered_data = data_categories.filter(lambda x: x.shape[0]> 50)

In [ ]:
filtered_category = filtered_data['medical_specialty'].value_counts()

In [ ]:
## Considering medical_specialty as traget variable and description and transcription as independent variables.
X = filtered_data['description']

In [ ]:
y = filtered_data['medical_specialty']

In [ ]:
import torch
from transformers import BertTokenizer
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
class BioBERTPreprocessor:
    def __init__(self, model_name='monologg/biobert_v1.1_pubmed', max_length=512):
        self.tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)
        self.max_length = max_length

    def clean_text(self, text_list):
        cleaned_data = []
        for text in text_list:
            cleaned_text = str(text).lower()  # Convert each element to lowercase string
            cleaned_data.append(cleaned_text)
        return cleaned_data

    def preprocess_data(self, text_data):
        cleaned_data = self.clean_text(text_data)
        tokenized_data = self.tokenizer.batch_encode_plus(
            cleaned_data,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',  # Enable padding to ensure sequences have the same length
            truncation=True,  # Enable truncation to handle sequences longer than max_length
            return_attention_mask=True,
            return_tensors='pt'
        )
        return tokenized_data

    def preprocess(self, X, y):
        # Clean and preprocess the independent data (X)
        tokenized_X = self.preprocess_data(X.values.tolist())

        # Prepare target variable (y)
        label_dict = {label: index for index, label in enumerate(y.unique())}
        y_encoded = y.map(label_dict)
        y_tensor = torch.tensor(y_encoded.values)

        return tokenized_X, y_tensor

# Example usage
# Initialize BioBERTPreprocessor
preprocessor = BioBERTPreprocessor()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

In [ ]:
# X = filtered_data[['description', 'transcription']]
# y = filtered_data['medical_specialty']

# Preprocess data
tokenized_X, y_tensor = preprocessor.preprocess(X, y)

# # Print tokenized data shape and example
# print("Tokenized data shape:", tokenized_X['input_ids'].shape)
# print("Example token IDs:", tokenized_X['input_ids'][0])
# print("Example attention mask:", tokenized_X['attention_mask'][0])
# print("Example segment IDs:", tokenized_X['token_type_ids'][0])
# print("Target variable shape:", y_tensor.shape)

In [ ]:
unique_labels = y.unique()

# Create label_dict mapping
label_dict = {label: index for index, label in enumerate(unique_labels)}

# Print label_dict
print(label_dict)

{' Cardiovascular / Pulmonary': 0, ' Neurology': 1, ' Urology': 2, ' General Medicine': 3, ' Surgery': 4, ' SOAP / Chart / Progress Notes': 5, ' Radiology': 6, ' Psychiatry / Psychology': 7, ' Pediatrics - Neonatal': 8, ' Pain Management': 9, ' Orthopedic': 10, ' Ophthalmology': 11, ' Office Notes': 12, ' Obstetrics / Gynecology': 13, ' Neurosurgery': 14, ' Nephrology': 15, ' Hematology - Oncology': 16, ' Gastroenterology': 17, ' ENT - Otolaryngology': 18, ' Emergency Room Reports': 19, ' Discharge Summary': 20, ' Consult - History and Phy.': 21}


In [ ]:

# Define model architecture (BioBERT fine-tuned for sequence classification)
model = BertForSequenceClassification.from_pretrained('monologg/biobert_v1.1_pubmed', num_labels=len(label_dict))

# Set device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/biobert_v1.1_pubmed and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(tokenized_X['input_ids'], y_tensor, test_size=0.2, random_state=42)
attention_mask_train, attention_mask_val = train_test_split(tokenized_X['attention_mask'], test_size=0.2, random_state=42)

In [ ]:
# Create DataLoader for training and validation sets
train_dataset = TensorDataset(X_train, attention_mask_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataset = TensorDataset(X_val, attention_mask_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
# Define optimizer and loss function
# Define optimizer using torch.optim.AdamW
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

In [ ]:
epochs = 1
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{epochs}, Average Training Loss: {avg_train_loss}')


Epoch 1/1, Average Training Loss: 2.343115760220422


In [ ]:
# # Save the model to the specified path

# model_save_path = '/content/sample_data/model.pth'
# torch.save(model.state_dict(), model_save_path)

In [ ]:
# ## Loading the model

# model1 = model.load_state_dict(torch.load(model_save_path))

In [ ]:
# Evaluation
model.eval()
val_preds = []
val_labels = []
with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

In [ ]:
# Calculate accuracy on validation set
val_accuracy = accuracy_score(val_labels, val_preds)
print(f'Validation Accuracy: {val_accuracy:.4f}')

Validation Accuracy: 0.4006
